## Imports

In [1]:
# data 
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

# modeling 
from sklearn.ensemble import RandomForestClassifier

# interactive 
from ipywidgets import interact, widgets  


## Data processing 

This process is shown in the Loan_Classification_Predictions notebook. This notebook is to show an interactive function. 

In [2]:
# laod data
df = pd.read_csv('Loan_Approval_Data_Set.csv') 
df = df.drop(columns=['Loan_ID'])


# dealing with na values 

df['Gender'].fillna(df['Gender'].value_counts().idxmax(), inplace=True)
df['Married'].fillna(df['Married'].value_counts().idxmax(), inplace=True)
df['Dependents'].fillna(df['Dependents'].value_counts().idxmax(), inplace=True)
df['Self_Employed'].fillna(df['Self_Employed'].value_counts().idxmax(), inplace=True)
df["LoanAmount"].fillna(df["LoanAmount"].mean(skipna=True), inplace=True)
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].value_counts().idxmax(), inplace=True)
df['Credit_History'].fillna(df['Credit_History'].value_counts().idxmax(), inplace=True)

df = pd.get_dummies(df)
df.drop(['Loan_Status_N'], axis = 1, inplace = True)

num_columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
df_no_scale = df[num_columns] 
df[num_columns] = MinMaxScaler().fit_transform(df[num_columns])

x_data = df.drop(columns='Loan_Status_Y')
y_data = df['Loan_Status_Y']

## Building the random forest 

In [3]:
rfc = RandomForestClassifier(criterion = 'entropy'
                             , max_features = 'sqrt'
                             , n_estimators = 200)

rfc.fit(x_data,y_data)

def yes_no_rfc(Applicant_Income, Coapplicant_Income,Loan_Amount
               , Loan_Amount_Term, Credit_History, Gender
               , Marital_Status, Dependents, Education_Graduate
               , Self_Employed, Property_Area): 
    # MinMaxScaler before being input for model 
    num_columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
    num_feature = [Applicant_Income, Coapplicant_Income, Loan_Amount, Loan_Amount_Term]
    df_pre_scale = df_no_scale.copy()
    df_pre_scale.loc[len(df)] = num_feature
    df_pre_scale[num_columns] = MinMaxScaler().fit_transform(df_pre_scale[num_columns])
    num_feature = df_pre_scale.iloc[-1:].values.flatten().tolist()
    
    # categorical features for predict 
    cat_feature = []
    
    if Credit_History == 'Yes': 
        cat_feature.append(1)
    else: 
        cat_feature.append(0)
    
    if Gender == 'Female':
        cat_feature.append(1)
        cat_feature.append(0)
    else:
        cat_feature.append(0)
        cat_feature.append(1)
    
    if Marital_Status == 'No':
        cat_feature.append(1)
        cat_feature.append(0)
    else: 
        cat_feature.append(0)
        cat_feature.append(1)
        
    if Dependents == 0:
        cat_feature.append(1)
        cat_feature.append(0)
        cat_feature.append(0)
        cat_feature.append(0)
    elif Dependents == 1: 
        cat_feature.append(0)
        cat_feature.append(1)
        cat_feature.append(0)
        cat_feature.append(0)
    elif Dependents == 2: 
        cat_feature.append(0)
        cat_feature.append(0)
        cat_feature.append(1)
        cat_feature.append(0)
    else: 
        cat_feature.append(0)
        cat_feature.append(0)
        cat_feature.append(0)
        cat_feature.append(1)  
    
    if Education_Graduate == 'Yes':
        cat_feature.append(1)
        cat_feature.append(0)
    else:
        cat_feature.append(0)
        cat_feature.append(1)
        
    if Self_Employed == 'No': 
        cat_feature.append(1)
        cat_feature.append(0)
    else:
        cat_feature.append(0)
        cat_feature.append(1)
        
    if Property_Area == 'Rural': 
        cat_feature.append(1)
        cat_feature.append(0)
        cat_feature.append(0)
    elif Property_Area == 'Semiurban': 
        cat_feature.append(0)
        cat_feature.append(1)
        cat_feature.append(0)
    else: 
        cat_feature.append(0)
        cat_feature.append(0)
        cat_feature.append(1)     
    
    # training model on all data 
    x = [num_feature + cat_feature]
    x =  pd.DataFrame(x, columns = list(df.columns)[:len(df.columns)-1])

    y = rfc.predict(x)
    
    if y == 1:
        print('You will be approved')
        
    else: 
        print('You will not be approved')
        

In [5]:
# style to keep function parameters visable in the widget 
style = {'description_width': 'initial'}

app_in_range = widgets.IntSlider(description = 'Applicant Income:', min = 0, max = 85000
                                 , value = 42500, style = style) # , layout={"width": "500px"})

coapp_in_range = widgets.IntSlider(description = 'Coapplicant Income:', min = 0, max = 85000
                                   , value = 42500, style = style)
la_range = widgets.IntSlider(description = 'Loan Amount:', min = 1, max = 800
                             , value = 400, style = style)
lat_range = widgets.IntSlider(description = 'Loan Amount Term:',min = 6, max = 480
                              , value = 243 ,style = style)
ch_ = widgets.Dropdown(description = 'Credit History:', options = ['Yes', 'No']
                       , value = 'Yes', style = style)
gender_ = widgets.Dropdown(description = 'Gender:',options = ['Male', 'Female']
                           , value = 'Male', style = style)
marr_ = widgets.Dropdown(description = 'Marital Status:', options = ['Yes', 'No']
                         , value = 'Yes', style = style)
depends_ = widgets.Dropdown(description = 'Dependents:', options = ['0', '1', '2', '3+']
                            , value = '0',style = style)
education_ = widgets.Dropdown(descriptiom = 'Education Graduate:', options = ['Yes', 'No']
                              , value = 'Yes', style = style)
selfe_ = widgets.Dropdown(description = 'Self Employed:', options = ['Yes', 'No']
                          , value = 'Yes', style = style)
prop = widgets.Dropdown(description = 'Property Area:', options = ['Rural', 'Semiurban', 'Urban']
                        , value = 'Rural', style = style)

yes_or_no = widgets.interact(yes_no_rfc, Applicant_Income = app_in_range
                    , Coapplicant_Income = coapp_in_range ,Loan_Amount = la_range
                    , Loan_Amount_Term = lat_range, Credit_History = ch_, Gender = gender_
                    , Marital_Status = marr_, Dependents = depends_ 
                    , Education_Graduate = education_, Self_Employed = selfe_
                    , Property_Area = prop)


interactive(children=(IntSlider(value=42500, description='Applicant Income:', max=85000, style=SliderStyle(des…